## Top 10 Films by rating
Spark Version 3.1.2
Databricks Runtime LTS 9.1

In [0]:
from pyspark.sql.functions import count, mean, desc

In [0]:
movies_filepath="dbfs:/FileStore/tables/asos_data/movies/"
rating_filepath="dbfs:/FileStore/tables/asos_data/rating/"
ouput_filepath="dbfs:/FileStore/tables/asos_delta_std/top10film/"

### Common Utility Functions

In [0]:
%run ../util/utils

In [0]:
####Getting Rating Schema from latest file path
rating_schema_filepath = get_latest_file_path(rating_filepath)
rating_schema = get_schema(rating_schema_filepath, "csv")

####Getting Rating Schema from latest file path
movies_schema_filepath = get_latest_file_path(movies_filepath)
movies_schema = get_schema(movies_schema_filepath, "csv")

### Loading csv files into pyspark dataframe

In [0]:
df_rating = spark.read.format("csv").schema(rating_schema) \
.option("header", True) \
.option("inferSchema", "true") \
.load(rating_filepath + "/*")

df_movies = spark.read.format("csv").schema(movies_schema) \
.option("header", True) \
.option("inferSchema", "true") \
.load(movies_filepath + "/*")

### Selecting the columns and performing join

In [0]:
cols_select = ["user_id", "movie_id", "movie_title", "rating"]
df_joined = df_rating.join(df_movies,"movie_id").select(*cols_select)

### Aggregation to calculate the top rating

In [0]:
df_final = df_joined.groupBy("movie_title") \
.agg(mean("rating").alias("toprating"), count("user_id").alias("count_userid")) \
.orderBy(desc("toprating"))

df_final = df_final.where(df_final['count_userid'] >= 5).limit(10)

In [0]:
display(df_final)

movie_title,toprating,count_userid
Pather Panchali (1955),4.625,8
"Close Shave, A (1995)",4.491071428571429,112
Schindler's List (1993),4.466442953020135,298
"Wrong Trousers, The (1993)",4.466101694915254,118
Casablanca (1942),4.45679012345679,243
Wallace & Gromit: The Best of Aardman Animation (1996),4.447761194029851,67
"Shawshank Redemption, The (1994)",4.445229681978798,283
Rear Window (1954),4.3875598086124405,209
"Usual Suspects, The (1995)",4.385767790262173,267
Star Wars (1977),4.3584905660377355,583


### Final output will be written into a single csv file

In [0]:
df_final.coalesce(1).write.mode("overwrite").option("header", True).csv(ouput_filepath)

### List of Output folder and display of the Output

In [0]:
display(dbutils.fs.ls(ouput_filepath))

path,name,size
dbfs:/FileStore/tables/asos_delta_std/top10film/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/tables/asos_delta_std/top10film/_committed_2710211864214872285,_committed_2710211864214872285,113
dbfs:/FileStore/tables/asos_delta_std/top10film/_started_2710211864214872285,_started_2710211864214872285,0
dbfs:/FileStore/tables/asos_delta_std/top10film/part-00000-tid-2710211864214872285-e39966ff-3c6b-4b42-bcae-de069a32bbd4-312-1-c000.csv,part-00000-tid-2710211864214872285-e39966ff-3c6b-4b42-bcae-de069a32bbd4-312-1-c000.csv,514


In [0]:
display(spark.read.format("csv").option("header", True).load(ouput_filepath))

movie_title,toprating,count_userid
Pather Panchali (1955),4.625,8
"Close Shave, A (1995)",4.491071428571429,112
Schindler's List (1993),4.466442953020135,298
"Wrong Trousers, The (1993)",4.466101694915254,118
Casablanca (1942),4.45679012345679,243
Wallace & Gromit: The Best of Aardman Animation (1996),4.447761194029851,67
"Shawshank Redemption, The (1994)",4.445229681978798,283
Rear Window (1954),4.3875598086124405,209
"Usual Suspects, The (1995)",4.385767790262173,267
Star Wars (1977),4.3584905660377355,583
